In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
import requests

lat = 24.8607  
lon = 67.0011

api_key = ""

url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"
response = requests.get(url)

print(response.status_code)
print(response.json())


200
{'coord': {'lon': 66.9983, 'lat': 24.8591}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 303.68, 'feels_like': 310.64, 'temp_min': 303.68, 'temp_max': 303.68, 'pressure': 998, 'humidity': 74, 'sea_level': 998, 'grnd_level': 996}, 'visibility': 10000, 'wind': {'speed': 5.17, 'deg': 251, 'gust': 5.86}, 'clouds': {'all': 100}, 'dt': 1751538840, 'sys': {'country': 'PK', 'sunrise': 1751503602, 'sunset': 1751552723}, 'timezone': 18000, 'id': 1174872, 'name': 'Karachi', 'cod': 200}


In [ ]:
import requests
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import pytz

# Set this to your Visual Crossing API key
VC_API_KEY = ''

def fetch_historical_weather(city, days=7):
    end_date = datetime.now().strftime('%Y-%m-%d')
    start_date = (datetime.now() - timedelta(days=days)).strftime('%Y-%m-%d')
    
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}/{start_date}/{end_date}?unitGroup=metric&include=days&key={VC_API_KEY}&contentType=json"
    response = requests.get(url)
    data = response.json()
    
    records = []
    for day in data['days']:
        records.append({
            'date': day['datetime'],
            'MinTemp': day['tempmin'],
            'MaxTemp': day['tempmax'],
            'Temp': day['temp'],
            'Humidity': day['humidity'],
            'Pressure': day['pressure'],
            'WindGustSpeed': day.get('windgust', day.get('windspeed', 0)),
            'Conditions': day.get('conditions', 'Unknown'),
            'RainTomorrow': 1 if 'rain' in day.get('conditions', '').lower() else 0
        })

    return pd.DataFrame(records)

def prepare_data(data):
    le = LabelEncoder()
    if 'Conditions' in data.columns:
        data['Conditions'] = le.fit_transform(data['Conditions'])
    X = data[['MinTemp', 'MaxTemp', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp', 'Conditions']]
    y = data['RainTomorrow']
    return X, y, le

def train_rain_model(X, y):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X, y)
    return model

def train_regression_model(X, y):
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y)
    return model

def prepare_regression_data(data, feature):
    X, y = [], []
    for i in range(len(data) - 1):
        X.append(data[feature].iloc[i])
        y.append(data[feature].iloc[i+1])
    return np.array(X).reshape(-1, 1), np.array(y)

def predict_future(model, current_value, steps=5):
    predictions = [current_value]
    for _ in range(steps):
        next_val = model.predict(np.array([[predictions[-1]]]))[0]
        predictions.append(next_val)
    return predictions[1:]

def get_current_weather(city):
    API_KEY = ""
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    return {
        'city': data['name'],
        'country': data['sys']['country'],
        'current_temp': data['main']['temp'],
        'feels_like': data['main']['feels_like'],
        'temp_min': data['main']['temp_min'],
        'temp_max': data['main']['temp_max'],
        'humidity': data['main']['humidity'],
        'description': data['weather'][0]['description'],
        'wind_speed': data['wind']['speed'],
        'pressure': data['main']['pressure'],
    }

def weather_view():
    city = input("Enter City Name: ")

    print("\nFetching historical weather data...")
    hist_df = fetch_historical_weather(city)

    if hist_df.empty or len(hist_df) < 5:
        print("Not enough historical data available.")
        return

    X, y, le = prepare_data(hist_df)
    rain_model = train_rain_model(X, y)

    # Regression models
    X_temp, y_temp = prepare_regression_data(hist_df, 'Temp')
    X_hum, y_hum = prepare_regression_data(hist_df, 'Humidity')
    temp_model = train_regression_model(X_temp, y_temp)
    hum_model = train_regression_model(X_hum, y_hum)

    print("\nFetching current weather data...")
    current = get_current_weather(city)

    # Prepare single-row DF for prediction
    condition_encoded = le.transform([current['description']])[0] if current['description'] in le.classes_ else -1
    X_live = pd.DataFrame([{
        'MinTemp': current['temp_min'],
        'MaxTemp': current['temp_max'],
        'WindGustSpeed': current['wind_speed'],
        'Humidity': current['humidity'],
        'Pressure': current['pressure'],
        'Temp': current['current_temp'],
        'Conditions': condition_encoded
    }])

    rain_pred = rain_model.predict(X_live)[0]
    future_temp = predict_future(temp_model, current['current_temp'])
    future_humidity = predict_future(hum_model, current['humidity'])

    # Future timestamps
    timezone = pytz.timezone('Asia/Karachi')
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)
    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:%M") for i in range(5)]

    # Output
    print(f"\nCity: {current['city']}, {current['country']}")
    print(f"Current Temp: {current['current_temp']}°C (Feels like {current['feels_like']}°C)")
    print(f"Humidity: {current['humidity']}%")
    print(f"Weather: {current['description']}")
    print(f"Rain Prediction: {'Yes ☔' if rain_pred == 1 else 'No ☀️'}")
    print("\nFuture Temp Predictions:")
    for time, t in zip(future_times, future_temp):
        print(f"{time}: {round(t, 1)}°C")
    print("\nFuture Humidity Predictions:")
    for time, h in zip(future_times, future_humidity):
        print(f"{time}: {round(h, 1)}%")

# Run it
weather_view()


Enter City Name:  San fransisco



Fetching historical weather data...

Fetching current weather data...

City: San Fransisco, EC
Current Temp: 7.93°C (Feels like 6.13°C)
Humidity: 71%
Weather: broken clouds
Rain Prediction: No ☀️

Future Temp Predictions:
17:00: 8.5°C
18:00: 8.6°C
19:00: 8.6°C
20:00: 8.6°C
21:00: 8.6°C

Future Humidity Predictions:
17:00: 78.1%
18:00: 75.0%
19:00: 70.3%
20:00: 71.5%
21:00: 78.1%
